Homework: Exercise 1: Threaded Prime Number Checker Write a Python program that checks whether a given range of numbers contains prime numbers. Divide the range among multiple threads to parallelize the prime checking process. Each thread should be responsible for checking a subset of the range, and the main program should print the list of prime numbers found.

In [2]:
import threading

# Function to check if a number is prime
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(n**0.5) + 1, 2):
        if n % i == 0:
            return False
    return True

# Thread worker function
def check_primes(start, end, result_list):
    primes = []
    for num in range(start, end + 1):
        if is_prime(num):
            primes.append(num)
    result_list.extend(primes)

# Main program
if __name__ == "__main__":
    start_range = 1
    end_range = 100
    num_threads = 4

    # Step size for each thread
    step = (end_range - start_range + 1) // num_threads

    threads = []
    results = []

    # Lock for safe list access (optional here)
    list_lock = threading.Lock()

    for i in range(num_threads):
        start = start_range + i * step
        end = start + step - 1

        # Ensure last thread covers the remainder
        if i == num_threads - 1:
            end = end_range

        # Create and start thread
        t = threading.Thread(
            target=lambda s=start, e=end: (
                list_lock.acquire(),
                results.extend([n for n in range(s, e + 1) if is_prime(n)]),
                list_lock.release()
            )
        )
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    # Sort results from all threads
    results.sort()

    print("Prime numbers found:")
    print(results)


Prime numbers found:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


Exercise 2: Threaded File Processing

Write a program that reads a large text file containing lines of text. Implement a threaded solution to count the occurrence of each word in the file. Each thread should process a portion of the file, and the main program should display a summary of word occurrences across all threads.

In [ ]:
import threading
from collections import Counter

# Thread worker function
def process_lines(lines, counter_list, lock):
    local_counter = Counter()

    for line in lines:
        words = line.strip().lower().split()
        local_counter.update(words)

    # Safely add local results to shared list
    with lock:
        counter_list.append(local_counter)

def threaded_word_count(filename, num_threads=4):
    # Read the entire file
    with open(filename, 'r', encoding='utf-8') as f:
        all_lines = f.readlines()

    total_lines = len(all_lines)
    step = total_lines // num_threads

    threads = []
    counters = []
    lock = threading.Lock()

    for i in range(num_threads):
        start = i * step
        end = start + step

        # Last thread gets remainder of lines
        if i == num_threads - 1:
            end = total_lines

        # Create thread
        t = threading.Thread(
            target=process_lines,
            args=(all_lines[start:end], counters, lock)
        )
        threads.append(t)
        t.start()

    # Wait for all threads to finish
    for t in threads:
        t.join()

    # Merge all Counters
    final_counter = Counter()
    for c in counters:
        final_counter.update(c)

    return final_counter


# ------------------------------
# Example Usage
# ------------------------------
if __name__ == "__main__":
    filename = "large_text.txt"   # your input file here

    results = threaded_word_count(filename, num_threads=4)

    print("Word Occurrences:")
    for word, count in results.most_common():
        print(f"{word}: {count}")
